In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import cudf
from tqdm import tqdm
tqdm.pandas()
import gc
from collections import Counter
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 200)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_path = Path("/kaggle/input/vtt-next-content-recommendation")

In [3]:
# item_domain filename is swapped
item_domain_1 = pd.read_csv(data_path/"item_domain_2 .csv")
item_domain_2 = pd.read_csv(data_path/"item_domain_1.csv")

train = pd.read_csv(data_path/"train.csv")
train.rename(columns={'ID_interaction':'session_id'}, inplace=True)

warm_up = pd.read_csv(data_path/"warm_up.csv")
warm_up.rename(columns={'ID_interaction':'session_id'}, inplace=True)

public_test = pd.read_csv(data_path/"public_test.csv")
public_test.rename(columns={'id_interaction':'session_id'}, inplace=True)

private_test = pd.read_csv(data_path/"private_test.csv")
private_test.rename(columns={'id_interaction':'session_id'}, inplace=True)

In [4]:
item_domain_1
item_domain_1.sort_values('published_time', ascending=False)

,video_id,duration,published_time,sub_category_id,raw_category_id,category_id
173584,1039682,1410,20230731233537,1558,1804.0,7
44565,1039691,1450,20230731230537,1558,1804.0,7
8089,1039679,1310,20230731223537,1558,1804.0,7
261841,1039577,674,20230731213637,"1561,1588",930.0,11
58158,1039676,821,20230731213537,1558,1804.0,7
...,...,...,...,...,...,...
260491,1051858,0,1,"1588,2407",248.0,18
39460,1057195,0,1,1558,1014.0,18
184061,1043992,0,1,5,299.0,18
156324,1049476,0,1,NaN,72.0,17


In [5]:
item_domain_2.sort_values('published_time', ascending=False)

,series_id,release_year,published_time,raw_category_id,director_id,actor_id,genre,category_id
3693,20518,2005.0,2023-09-23 06:00:00.0,3467,38402,"29039,2339,104587,38393,38396,38399,41852",drama,5
4046,20515,2017.0,2023-09-14 06:00:00.0,3470,2642,NaN,comedy,3
1959,20536,1995.0,2023-09-01 06:00:00.0,3398,34343,"10751,2681,104638,104635,104632,30974,20224",action,1
4185,20524,2007.0,2023-09-01 06:00:00.0,395,104617,"104605,104608,104611,104614,104602,104251,4920...",drama,5
4031,20539,1989.0,2023-09-01 06:00:00.0,3398,47114,"22096,26191,104638,104641,104632,104644,104647",action,1
...,...,...,...,...,...,...,...,...
1054,4151,2014.0,2017-08-17 08:04:11.0,"1122,1137",12338,"12335,8903",china,2
2773,851,2015.0,2017-08-02 18:24:58.0,"5,1065,1128",1145,"1136,1139,1142",comedy|usuk,3
4907,842,2014.0,2017-07-17 22:47:34.0,"5,1065,1137",1118,"1106,1109,1112,1115",comedy|china,3
1595,596,2014.0,2017-03-07 10:18:20.0,1128,389,386,usuk,24


In [6]:
train

,session_id,list_interaction
0,0,1_311485|2_16174|2_18058|1_932179|1_256648|1_3...
1,1,1_945706|2_18196|1_945712|1_937924|1_939310|2_...
2,2,1_189193|1_734116|2_17684
3,3,1_931450|2_18130|1_570697|2_17077|1_931483|2_1...
4,4,1_931432|1_925399|1_381490|1_610526|1_810136|1...
...,...,...
1293536,1293536,2_14219|2_17447|2_16558|2_16678
1293537,1293537,1_852337|1_852334|2_5821|2_17854|1_696922|2_17...
1293538,1293538,2_7363|2_9586|1_941866|2_17917|1_932413|2_1865...
1293539,1293539,1_852334|2_16393|2_17675|1_838471|1_854518


In [7]:
warm_up

,session_id,list_interaction
0,1,2_17201|2_18433|1_942670|2_18745|2_18817|1_960619
1,2,2_10477|2_17255|2_18025|2_17471|2_18610|2_7561
2,3,1_899363|2_6407|2_17944|1_729940|2_15544|1_293...
3,4,2_17131|2_17941|1_950500|1_935275|2_7183|1_958...
4,5,2_18862|2_11087|2_18040|2_9103|2_18634|1_95041...
...,...,...
9995,9996,2_9052|2_7567|2_18607|2_18505|2_15475|2_18529|...
9996,9997,2_16048|2_6616|2_18058|1_956839|1_956842|1_955...
9997,9998,1_966376|1_968269|1_955807|1_965638|2_18151|1_...
9998,9999,2_11261|1_968728|1_968077|1_968731|1_969562|1_...


In [8]:
public_test

,session_id,list_interaction
0,1,2_17917|1_947551|1_954310|1_729940|1_421528|1_...
1,2,1_957928|1_967606|1_963436|1_964528|1_964807|1...
2,3,1_962968|2_6892|1_940723|1_961114|2_7507|2_789...
3,4,2_18457|1_969562|1_969877|2_18952|2_16393|1_96...
4,5,2_18397|2_17917|2_18196|2_18007|2_18079|2_1806...
...,...,...
112157,112158,1_955957|1_805597|1_960052|1_959353|2_7573|1_9...
112158,112159,2_10510|1_967606|2_6892|2_19135|2_15466|1_9533...
112159,112160,1_854788|1_968410|1_968416|1_966916|1_961063|1...
112160,112161,2_15958|2_6892|2_18151|2_10525|2_18376|2_10678...


In [9]:
private_test

,session_id,list_interaction
0,100538773,1_813844|1_961681|2_18709|2_19870|2_19603|2_19...
1,100898026,1_1045654|1_1030345|1_805597|1_1020751|2_19150...
2,101660095,1_981172|1_1051768|1_1051354|1_1051396|1_1051723
3,101686687,2_17077|2_19642|2_20318|2_17258|2_20290
4,101765287,1_1053289|2_19642|1_1040174|1_1048963|1_104847...
...,...,...
147730,184219358,2_18664|2_17734|2_19075|1_964237|2_17941|2_776...
147731,184629737,1_969073|1_968731|1_971302|2_17672|1_990715|2_...
147732,185328920,2_17077|1_955807|1_975355|1_982300|1_697087|1_...
147733,185356775,2_18433|2_18664|2_6769|2_18487|1_974596|2_5803


In [10]:
test = private_test.copy()
train = pd.concat([train, warm_up, public_test]).drop_duplicates(subset=['list_interaction'])
train = train[~train['list_interaction'].isin(test['list_interaction'])].reset_index(drop=True)
train['session_id'] = np.arange(len(train))

In [11]:
%%time
def f(list_interaction: str, prefix: str):
    seq = []
    for item in list_interaction.split('|'):
        if item[:2] == prefix:
            seq.append(item[2:])
    return seq

train['domain1_seq'] = train['list_interaction'].map(lambda s: f(s, prefix='1_'))
train['domain2_seq'] = train['list_interaction'].map(lambda s: f(s, prefix='2_'))
test['domain1_seq'] = test['list_interaction'].map(lambda s: f(s, prefix='1_'))
test['domain2_seq'] = test['list_interaction'].map(lambda s: f(s, prefix='2_'))


train['domain1_len'] = train['domain1_seq'].map(lambda s: len(s))
train['domain2_len'] = train['domain2_seq'].map(lambda s: len(s))
test['domain1_len'] = test['domain1_seq'].map(lambda s: len(s))
test['domain2_len'] = test['domain2_seq'].map(lambda s: len(s))

train['domain1_truncated'] = train['domain1_seq'].map(lambda s: s[-50:])
train['domain2_truncated'] = train['domain2_seq'].map(lambda s: s[-50:])
test['domain1_truncated'] = test['domain1_seq'].map(lambda s: s[-50:])
test['domain2_truncated'] = test['domain2_seq'].map(lambda s: s[-50:])

CPU times: user 27.1 s, sys: 1.6 s, total: 28.7 s
Wall time: 28.7 s


In [12]:
test

,session_id,list_interaction,domain1_seq,domain2_seq,domain1_len,domain2_len,domain1_truncated,domain2_truncated
0,100538773,1_813844|1_961681|2_18709|2_19870|2_19603|2_19...,"[813844, 961681, 957688, 956824, 982300, 10464...","[18709, 19870, 19603, 19648, 17179, 20365, 190...",20,27,"[813844, 961681, 957688, 956824, 982300, 10464...","[18709, 19870, 19603, 19648, 17179, 20365, 190..."
1,100898026,1_1045654|1_1030345|1_805597|1_1020751|2_19150...,"[1045654, 1030345, 805597, 1020751, 985720, 63...","[19150, 17273, 17158, 18517, 10972, 16117, 169...",67,55,"[1043926, 1022161, 1039109, 1038460, 943810, 7...","[16117, 16942, 19612, 4283, 19765, 16555, 1852..."
2,101660095,1_981172|1_1051768|1_1051354|1_1051396|1_1051723,"[981172, 1051768, 1051354, 1051396, 1051723]",[],5,0,"[981172, 1051768, 1051354, 1051396, 1051723]",[]
3,101686687,2_17077|2_19642|2_20318|2_17258|2_20290,[],"[17077, 19642, 20318, 17258, 20290]",0,5,[],"[17077, 19642, 20318, 17258, 20290]"
4,101765287,1_1053289|2_19642|1_1040174|1_1048963|1_104847...,"[1053289, 1040174, 1048963, 1048477, 1049179, ...","[19642, 20336, 20149, 18448, 19174, 16288]",33,6,"[1053289, 1040174, 1048963, 1048477, 1049179, ...","[19642, 20336, 20149, 18448, 19174, 16288]"
...,...,...,...,...,...,...,...,...
147730,184219358,2_18664|2_17734|2_19075|1_964237|2_17941|2_776...,"[964237, 984367]","[18664, 17734, 19075, 17941, 7768, 17077, 1903...",2,12,"[964237, 984367]","[18664, 17734, 19075, 17941, 7768, 17077, 1903..."
147731,184629737,1_969073|1_968731|1_971302|2_17672|1_990715|2_...,"[969073, 968731, 971302, 990715]","[17672, 9040]",4,2,"[969073, 968731, 971302, 990715]","[17672, 9040]"
147732,185328920,2_17077|1_955807|1_975355|1_982300|1_697087|1_...,"[955807, 975355, 982300, 697087, 936604, 70141...","[17077, 16600]",8,2,"[955807, 975355, 982300, 697087, 936604, 70141...","[17077, 16600]"
147733,185356775,2_18433|2_18664|2_6769|2_18487|1_974596|2_5803,[974596],"[18433, 18664, 6769, 18487, 5803]",1,5,[974596],"[18433, 18664, 6769, 18487, 5803]"


In [13]:
c1 = Counter(chain(*[[int(i) for i in s] for s in train['domain1_seq'].values]))
c2 = Counter(chain(*[[int(i) for i in s] for s in test['domain1_seq'].values]))

# Create covisitation matrix

In [14]:
train.head()

,session_id,list_interaction,domain1_seq,domain2_seq,domain1_len,domain2_len,domain1_truncated,domain2_truncated
0,0,1_311485|2_16174|2_18058|1_932179|1_256648|1_3...,"[311485, 932179, 256648, 309004, 733129, 93231...","[16174, 18058, 17923, 17848, 8617, 16030, 6670...",41,11,"[311485, 932179, 256648, 309004, 733129, 93231...","[16174, 18058, 17923, 17848, 8617, 16030, 6670..."
1,1,1_945706|2_18196|1_945712|1_937924|1_939310|2_...,"[945706, 945712, 937924, 939310, 943483, 94502...","[18196, 17923, 17941, 7585, 8617, 18529, 17959...",135,22,"[932377, 945679, 891010, 290224, 932386, 93915...","[18196, 17923, 17941, 7585, 8617, 18529, 17959..."
2,2,1_189193|1_734116|2_17684,"[189193, 734116]",[17684],2,1,"[189193, 734116]",[17684]
3,3,1_931450|2_18130|1_570697|2_17077|1_931483|2_1...,"[931450, 570697, 931483, 320392, 933439]","[18130, 17077, 15265, 10402, 18253, 17977]",5,6,"[931450, 570697, 931483, 320392, 933439]","[18130, 17077, 15265, 10402, 18253, 17977]"
4,4,1_931432|1_925399|1_381490|1_610526|1_810136|1...,"[931432, 925399, 381490, 610526, 810136, 26449...","[17149, 18373, 18385, 18427, 16438]",51,5,"[925399, 381490, 610526, 810136, 264496, 93142...","[17149, 18373, 18385, 18427, 16438]"


In [15]:
train['have_duplicated_1'] = train['domain1_seq'].map(lambda x: len(set(x)) != len(x))
train['have_duplicated_2'] = train['domain2_seq'].map(lambda x: len(set(x)) != len(x))
print(train['have_duplicated_1'].sum(), train['have_duplicated_2'].sum())

0 0


* No duplicated item in domain_1 or domain_2 sequence --> post-processing --> lb score hurt

In [16]:
df1 = pd.concat([train[train['domain1_len'] >= 2][['domain1_truncated']], 
                 test[test['domain1_len'] >= 2][['domain1_truncated']]], axis=0).reset_index(drop=True)
df1['session_id'] = df1.index.astype('int32')

df2 = pd.concat([train[train['domain2_len'] >= 2][['domain2_truncated']], 
                 test[test['domain2_len'] >= 2][['domain2_truncated']]], axis=0).reset_index(drop=True)
df2['session_id'] = df2.index.astype('int32')

In [17]:
df1

,domain1_truncated,session_id
0,"[311485, 932179, 256648, 309004, 733129, 93231...",0
1,"[932377, 945679, 891010, 290224, 932386, 93915...",1
2,"[189193, 734116]",2
3,"[931450, 570697, 931483, 320392, 933439]",3
4,"[925399, 381490, 610526, 810136, 264496, 93142...",4
...,...,...
789325,"[985882, 982720, 983227, 993808, 984283, 98282...",789325
789326,"[964237, 984367]",789326
789327,"[969073, 968731, 971302, 990715]",789327
789328,"[955807, 975355, 982300, 697087, 936604, 70141...",789328


In [18]:
df2

,domain2_truncated,session_id
0,"[16174, 18058, 17923, 17848, 8617, 16030, 6670...",0
1,"[18196, 17923, 17941, 7585, 8617, 18529, 17959...",1
2,"[18130, 17077, 15265, 10402, 18253, 17977]",2
3,"[17149, 18373, 18385, 18427, 16438]",3
4,"[18013, 18199, 17684, 17926]",4
...,...,...
1211465,"[18664, 17734, 19075, 17941, 7768, 17077, 1903...",1211465
1211466,"[17672, 9040]",1211466
1211467,"[17077, 16600]",1211467
1211468,"[18433, 18664, 6769, 18487, 5803]",1211468


In [19]:
DISK_PIECES = 50
def create_covisitation(df, interaction_col, domain_id, top_k=50):
    covisit = dict()
    df = cudf.DataFrame(df.explode(interaction_col).rename(columns={interaction_col : 'item_id'}))
    df['item_id'] = df['item_id'].astype('int32')
    df = df.merge(df, on='session_id')
    df = df.loc[df['item_id_x'] != df['item_id_y']]
    SIZE = (df['item_id_x'].max() // DISK_PIECES) + 1
#     print(df.shape)
    for PART in range(DISK_PIECES):
        # MEMORY MANAGEMENT COMPUTE IN PARTS
        tmp = df.loc[(df['item_id_x'] >= PART*SIZE)&(df['item_id_x'] < (PART+1)*SIZE)]
        tmp = tmp[['item_id_x', 'item_id_y']]
        tmp = tmp.groupby(['item_id_x', 'item_id_y']).size().to_frame('weight').reset_index()
        tmp = tmp.sort_values(['item_id_x', 'weight'], ascending=[True, False])
        tmp['n'] = tmp.groupby('item_id_x')['item_id_y'].cumcount()
        tmp = tmp.loc[tmp['n'] < top_k].drop('n', axis=1)
        covisit_part = tmp.to_pandas().groupby('item_id_x')['item_id_y'].apply(list).to_dict()
        covisit.update(covisit_part)
    del tmp, covisit_part, df 
    gc.collect()
    
    return covisit
    

In [20]:
covisit_1 = create_covisitation(df1, interaction_col='domain1_truncated', domain_id='1')
covisit_2 = create_covisitation(df2, interaction_col='domain2_truncated', domain_id='2')
len(covisit_1) / len(item_domain_1), len(covisit_2) / len(item_domain_2)

(0.17179710061637046, 0.6523745433570467)

In [21]:
# use both train and test dataset --> worse than using test only --> data time shift
data = pd.concat([train[['domain1_len', 'domain1_seq', 'domain2_len', 'domain2_seq']], 
                  test[['domain1_len', 'domain1_seq', 'domain2_len', 'domain2_seq']]])
c1 = Counter(chain(*[[int(i) for i in s] for s in test['domain1_seq'].values]))
c2 = Counter(chain(*[[int(i) for i in s] for s in test['domain2_seq'].values]))
print(len(c1), len(c2))

40835 2632


In [22]:
# Cold user with only one item interaction counter
c3 = Counter(chain(*[[int(i) for i in s] for s in test[test['domain1_len'] == 1]['domain1_seq'].values]))
c4 = Counter(chain(*[[int(i) for i in s] for s in test[test['domain2_len'] == 1]['domain2_seq'].values]))
print(len(c3), len(c4))

2503 934


In [23]:
def predict(seq, domain_id, domain_counter, covisit, cold_user_counter, k=50):
    if len(seq) >= k:
        items = [int(v) for v in seq]
        preds = items[:k] # use head >> tail by 0.000593
        return '|'.join([f"{domain_id}_{v}" for v in preds])
    elif len(seq) > 0 and len(seq) < k:
        clicked_items = [int(v) for v in seq]
        # USE "CLICKS" CO-VISITATION MATRIX
        items = list(chain(*[covisit[item] for item in clicked_items if item in covisit]))
        # RERANK CANDIDATES
        top_items = [item for item, cnt in Counter(items).most_common(k) if item not in clicked_items]
        preds = clicked_items + top_items[:(k - len(clicked_items))]
        # USE TOP 50 TEST CLICKS
        if len(preds) < k:
            preds += [item for item, cnt in domain_counter.most_common(k) if item not in preds][:k]
        return '|'.join([f"{domain_id}_{item}" for item in preds])
    return '|'.join([f"{domain_id}_{item}" for item, cnt in cold_user_counter.most_common(k)])

In [24]:
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=2)

  Preparing metadata (setup.py) ... - done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16677 sha256=d2d9947d93a2c11776e9ac786b5d4b8b078d25c3e94536388598ebe7c7f82f2a
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [25]:
%%time
test['next_item_1'] = test['domain1_seq'].parallel_map(lambda x: predict(x, domain_id='1', domain_counter=c1, 
                                                                                       cold_user_counter=c3, covisit=covisit_1))
test['next_item_2'] = test['domain2_seq'].parallel_map(lambda x: predict(x, domain_id='2', domain_counter=c2, 
                                                                                       cold_user_counter=c4, covisit=covisit_2))

CPU times: user 2.03 s, sys: 968 ms, total: 2.99 s
Wall time: 40.9 s


In [26]:
sub = test[['session_id', 'list_interaction', 'next_item_1', 'next_item_2']]
sub.columns=['ID_interaction', 'list_interaction', 'next_item_1', 'next_item_2']
sub.drop(columns=['list_interaction'], inplace=True)
compression_opts = dict(method='zip', archive_name='results.csv')  
sub.to_csv('results.zip', compression=compression_opts)

In [27]:
sub

,ID_interaction,next_item_1,next_item_2
0,100538773,1_813844|1_961681|1_957688|1_956824|1_982300|1...,2_18709|2_19870|2_19603|2_19648|2_17179|2_2036...
1,100898026,1_1045654|1_1030345|1_805597|1_1020751|1_98572...,2_19150|2_17273|2_17158|2_18517|2_10972|2_1611...
2,101660095,1_981172|1_1051768|1_1051354|1_1051396|1_10517...,2_19012|2_20281|2_19669|2_9040|2_19174|2_20191...
3,101686687,1_982300|1_990715|1_784300|1_1023391|1_925921|...,2_17077|2_19642|2_20318|2_17258|2_20290|2_9040...
4,101765287,1_1053289|1_1040174|1_1048963|1_1048477|1_1049...,2_19642|2_20336|2_20149|2_18448|2_19174|2_1628...
...,...,...,...
147730,184219358,1_964237|1_984367|1_967009|1_952684|1_963313|1...,2_18664|2_17734|2_19075|2_17941|2_7768|2_17077...
147731,184629737,1_969073|1_968731|1_971302|1_990715|1_969070|1...,2_17672|2_9040|2_17881|2_18199|2_17207|2_8917|...
147732,185328920,1_955807|1_975355|1_982300|1_697087|1_936604|1...,2_17077|2_16600|2_18199|2_18196|2_17881|2_1819...
147733,185356775,1_974596|1_982300|1_979570|1_974533|1_885406|1...,2_18433|2_18664|2_6769|2_18487|2_5803|2_17881|...


In [28]:
sanity_check1 = (sub['next_item_1'].map(lambda x: len(x.split('|'))) < 50).sum()
sanity_check2 = (sub['next_item_2'].map(lambda x: len(x.split('|'))) < 50).sum()
print(sanity_check1, sanity_check2)

0 0
